In [1]:
import os, sys
import pandas as pd
import geopandas as gpd
import rasterio as rio
import numpy as np


In [ ]:
import geowombat as gw
import shutil
in_dir="/home/l_sharwood/images/"
# done_fis = ["S2A_34HBH_20151218_0_L2A.tif" ,  "S2A_34HBH_20151228_0_L2A.tif", "S2A_34HBH_20160117_0_L2A.tif", "S2A_34HBH_20160216_0_L2A.tif", 'S2A_34HBH_20160307_0_L2A.tif', "S2A_34HBH_20160725_0_L2A.tif", "S2A_34HBH_20160526_0_L2A.tif", "S2A_34HBH_20160605_0_L2A.tif", "S2A_34HBH_20160804_0_L2A.tif", "S2A_34HBH_20160903_0_L2A.tif", "S2A_34HBH_20160923_0_L2A", "S2A_34HBH_20161003_0_L2A", "S2A_34HBH_20161013_0_L2A", "S2A_34HBH_20161023_0_L2A", "S2A_34HBH_20161122_0_L2A", "S2A_34HBH_20161202_0_L2A"]
# out_rasts = [i for i in os.listdir(in_dir) if (i.endswith(".tif") and i not in done_fis)]

toDos=[ 
    # "S2A_34HBH_20151218_0_L2A.tif", "S2A_34HBH_20151228_0_L2A.tif",
    #    "S2A_34HBH_20160117_0_L2A.tif", 
    #    "S2A_34HBH_20160216_0_L2A.tif", 
    #    'S2A_34HBH_20160307_0_L2A.tif', 
         "S2A_34HBH_20160406_0_L2A.tif", 
    # "S2A_34HBH_20160406_1_L2A.tif", 
    #    "S2A_34HBH_20160526_0_L2A.tif", 
    #    "S2A_34HBH_20160605_0_L2A.tif", 
    #    "S2A_34HBH_20160725_0_L2A.tif", 
    #    "S2A_34HBH_20160804_0_L2A.tif", 
      #  "S2A_34HBH_20160903_0_L2A.tif", 
    # "S2A_34HBH_20160923_0_L2A.tif", 
    #    "S2A_34HBH_20161003_0_L2A.tif", "S2A_34HBH_20161013_0_L2A.tif", "S2A_34HBH_20161023_0_L2A.tif",
    #    "S2A_34HBH_20161122_0_L2A.tif", 
    #    "S2A_34HBH_20161202_0_L2A.tif"
] 
out_rasts = sorted([i for i in os.listdir(in_dir) if (i.endswith(".tif") and i in toDos)])
print(out_rasts)

grid_gdf = gpd.read_file("/home/l_sharwood/code/cape_gridBuf_utm32S.shp")
unq_list = [1,2,5,6,7,8,9,10,11,13,14,15,17,18,19,20] #,
grids_totile=grid_gdf[grid_gdf['UNQ'].isin(unq_list)]


for k, v in grids_totile.iterrows():
    UNQ = v['UNQ']
    QC_db = "/home/sandbox-cel/capeTown/stac_grids/"+str(UNQ).zfill(6)+"/sentinel2/scene.info"
    out_dir = os.path.join("/home/sandbox-cel/capeTown/stac_grids", str(UNQ).zfill(6), "sentinel2")
    bounds = v.geometry.bounds
    ## out_rasts = [i for i in os.listdir(in_dir) if (i.endswith(".tif") and i in done_fis)]
    for rast in out_rasts:
        full_out_name = os.path.join(out_dir, rast)
        meta_file=full_out_name[:-4]+"_metadata.xml"
        name=os.path.basename(meta_file)[:-13]
        date = pd.to_datetime(name.split("_")[2], format='%Y%m%d')
        print(name, date)
        
        shutil.copyfile(os.path.join(in_dir, rast[:-4]+".xml"), meta_file)
        # if os.path.exists(os.path.join(out_dir, rast)):
        #     os.remove(os.path.join(out_dir, rast))
        print(meta_file)
        with gw.config.update(ref_crs=32734,  ref_res=10, ref_bounds=bounds):
            with gw.open(os.path.join(in_dir, rast), chunks=1024) as ref:
                attrs = ref.attrs.copy()
                ref = (ref).assign_attrs(**attrs, verbose=1)
                ref.gw.to_raster(os.path.join(out_dir, rast), indexes=6, n_workers=4, n_threads=1)                
                print(os.path.join(out_dir, rast))   
                
                
        if os.path.exists(QC_db):
            p_df = pd.read_pickle(QC_db)
            p_df.drop(p_df.loc[[i.split("_")[-2] == '1' for i in p_df['id']] ].index, inplace=True)
           ## p_df.drop(p_df.loc[[i.year <= 2017 for i in p_df['date']] ].index, inplace=True)
            p_df.loc[len(p_df.index)] = [name, 
                                         meta_file, 
                                         date]
        display(p_df.drop_duplicates().sort_values('date'))
        p_df.drop_duplicates().sort_values('date').to_pickle(QC_db)


In [ ]:
import os
import pandas as pd
import math

for grid in [1,2,5,6,7,8,9,10,11,13,14,15,17,18,19,20]:
    print(grid)
    monthDIR = os.path.join("/home/sandbox-cel/capeTown/monthly/sentinel2_brdf", str(grid).zfill(2))
    rm_files = [i for i in os.listdir(monthDIR) if ("_201604" in i)]
    print(rm_files)
    %cd $monthDIR
    for file in rm_files:
        !rm $file

In [66]:
## BRDF
## monthly greenness
## mosaic
## decile monthly

In [151]:

from osgeo import gdal
import os

def monthly_vrts(month_dir, VI, mosaic_dir):
    
    if "landsat" in month_dir:
        sensor="LC"
    elif "sentinel2" in month_dir:
        sensor="S2"    

    if not os.path.exists(mosaic_dir):
        os.makedirs(mosaic_dir)
        
    monthly_mosaics = []
    for year in list(range(2016, 2022, 1)):
        for month in list(range(1, 13, 1)):
            date = str(year)+str(month).zfill(2)
            grids = sorted([os.path.join(month_dir, g) for g in os.listdir(month_dir) if len(g) == 2 ])
            files_to_mos = []
            for grid_dir in grids:
                file2mos = sorted([os.path.join(grid_dir, file) for file in os.listdir(grid_dir) if date in file and VI+"_20" in file])
                if len(file2mos) == 1:
                    files_to_mos.append(file2mos[0])
                else:
                    print(VI, grid_dir, date, len(file2mos))
            out_vrt=os.path.join(mosaic_dir, sensor+"_"+str(VI)+"_"+str(date)+".vrt")
            gdal.BuildVRT(out_vrt, sorted(files_to_mos))
            monthly_mosaics.append(out_vrt)
            
    return monthly_mosaics

In [ ]:
import os
from osgeo import gdal

month_dir ='/home/sandbox-cel/capeTown/monthly/sentinel2_brdf'
mosaic_dir = '/home/sandbox-cel/capeTown/monthly/sentinel2_brdf/vrts'

if not os.path.exists(mosaic_dir):
    os.makedirs(mosaic_dir)
for VI in ['savi', 'evi', 'ndvi', 'ndmi1', 'ndmi2']: ##'evi', 'ndvi', 'ndmi1', 'ndmi2', 'ndwi', 'ndwi_rev'
    monthly_vrts(month_dir=month_dir, VI=VI, mosaic_dir=mosaic_dir)
    
#10,14,15,18,19

In [ ]:
unq05dates=UNQ_date_dict['05']
unq01dates=UNQ_date_dict['01']
sorted([i for i in unq05dates if i not in unq01dates])